## 1. Import Python Libraries

In [1]:
import boto3, cv2, time, numpy as np, matplotlib.pyplot as plt, random
import base64, json

## 2. Check if Endpoint creation is successful and create the predictor

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

# Restore the endpoint name stored in the 2_DeployEndpoint.ipynb notebook
%store -r ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

endpoint_created = False
while True:
    response = sm_client.list_endpoints()
    for ep in response['Endpoints']:
        print(f"Endpoint Status = {ep['EndpointStatus']}")
        if ep['EndpointName']==ENDPOINT_NAME and ep['EndpointStatus']=='InService':
            endpoint_created = True
            break
    if endpoint_created:
        break
    time.sleep(5)

## 3. Run Inference and Generate output results

In [ ]:
infer_start_time = time.time()

# Read the image into a numpy  array
orig_image = cv2.imread('images-test/z6686604803522_8d40b41b8489d82e13f15d9db092d1d8.jpg')

# Conver the array into jpeg
jpeg = cv2.imencode('.jpg', orig_image)[1]
# Serialize the jpg using base 64
payload = base64.b64encode(jpeg).decode('utf-8')

runtime= boto3.client('runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName=ENDPOINT_NAME,
                                        ContentType='text/csv',
                                        Body=payload)
response_body = response['Body'].read()
result = json.loads(response_body.decode('ascii'))

infer_end_time = time.time()

print(f"Inference Time = {infer_end_time - infer_start_time:0.4f} seconds")

print(result)


## 4. Print and Generate image

In [ ]:
CLASS_COLORS = {
    0: (0,255,0),
    1: (255,0,0),
    2: (0,0,255),
    3: (0,0,0),
}

CLASS_NAMES = {
    0: "abben",
    1: "boncha",
    2: "joco",
    3: "shelf",
}

for box in result['boxes']:
    x1, y1, x2, y2, conf, classID = box
    color = CLASS_COLORS.get(int(classID), (255,255,255))
    # label = f"{CLASS_NAMES.get(classID, classID)} {conf:.2f}"
    # Draw rectangle
    cv2.rectangle(orig_image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness=2)
    # # Draw label background
    # (w, h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    # cv2.rectangle(img, (int(x1), int(y1) - h - 4),
    #                    (int(x1) + w, int(y1)), color, -1)
    # # Put text
    # cv2.putText(img, label, (int(x1), int(y1) - 2),
    # cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 1, cv2.LINE_AA)


plt.figure(figsize=(12,8))
plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

out_path = 'output_with_boxes.jpg'
cv2.imwrite(out_path, orig_image)
print(f"Annotated image saved to {out_path}")
